Mover las imagenes

In [3]:
import os
import shutil

# Rutas de las carpetas
carpeta_A = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/images"
carpeta_B = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Gatos"

# Extensiones de imágenes a mover
extensiones = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp")

contador = 0
for raiz, carpetas, archivos in os.walk(carpeta_A):
    for archivo in archivos:
        if archivo.lower().endswith(extensiones):
            origen = os.path.join(raiz, archivo)
            destino = os.path.join(carpeta_B, archivo)

            if os.path.exists(destino):
                nombre, ext = os.path.splitext(archivo)
                i = 1
                while os.path.exists(destino):
                    nuevo_nombre = f"{nombre}_{i}{ext}"
                    destino = os.path.join(carpeta_B, nuevo_nombre)
                    i += 1

            shutil.move(origen, destino)
            contador += 1

            # Solo mostrar mensaje cada 100 imágenes
            if contador % 100 == 0:
                print(f"Movidas {contador} imágenes...", end="\r")

print(f"Se movieron {contador} imágenes correctamente.")


Se movieron 126607 imágenes correctamente.


Eliminar algunas imagenes de forma uniforme

In [4]:
import os
import math

# Configuracion
carpeta = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Gatos"
total_deseado = 22000
extensiones = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

# Lista de imagenes
imagenes = [os.path.join(carpeta, f) for f in os.listdir(carpeta)
             if f.lower().endswith(extensiones)]
imagenes.sort()  

total_actual = len(imagenes)
print(f"Total actual: {total_actual}")

# Calcular Imagenes
if total_actual <= total_deseado:
    print("Ya tienes menos o igual a 20k imágenes, no se elimina nada.")
else:
    paso = total_actual / total_deseado
    print(f"Se conservará 1 de cada {paso:.2f} imágenes aproximadamente.")

    # --- Seleccionar imagenes a conservar
    indices_conservar = {math.floor(i * paso) for i in range(total_deseado)}
    
    # Eliminar el resto
    eliminadas = 0
    for idx, ruta in enumerate(imagenes):
        if idx not in indices_conservar:
            os.remove(ruta)
            eliminadas += 1
            if eliminadas % 1000 == 0:
                print(f"Eliminadas {eliminadas} imágenes...", end="\r")

    print(f"\n Eliminadas {eliminadas} imágenes. Quedan {total_deseado} en total.")


Total actual: 126607
Se conservará 1 de cada 5.75 imágenes aproximadamente.
🗑️  Eliminadas 104000 imágenes...
 Eliminadas 104607 imágenes. Quedan 22000 en total.


Extraer imagenes

In [2]:
import joblib
import numpy as np
from PIL import Image
import os

# Nombre del archivo .sav que descargaste
archivo_sav = '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/turtles128.sav'

# Carpeta donde quieres guardar las imágenes
carpeta_salida = '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'

#Crear la carpeta de salida si no existe
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)
    print(f"Carpeta '{carpeta_salida}' creada.")

#Cargar el archivo .sav
print(f"Cargando '{archivo_sav}'... (Esto puede tardar un momento)")
lista_de_imagenes = joblib.load(archivo_sav)
print(f"¡Archivo cargado! Se encontraron {len(lista_de_imagenes)} imágenes.")

#Convertir la lista a un array de NumPy
dataset_array = np.array(lista_de_imagenes)

print(f"Guardando imágenes en '{carpeta_salida}'...")
for i, img_array in enumerate(dataset_array):
    

    #Escalamos de [0, 1] a [0, 255] y convertimos a entero
    img_array_uint8 = (img_array * 255.0).astype(np.uint8)

    # Crear un objeto de imagen desde el array de píxeles
    img = Image.fromarray(img_array_uint8)
    
    # Definir el nombre del archivo (ej: tortuga_00001.png)
    nombre_archivo = f'tortuga_{str(i).zfill(5)}.png'
    ruta_archivo = os.path.join(carpeta_salida, nombre_archivo)
    
    # Guardar la imagen
    img.save(ruta_archivo)

print(f"¡Listo! Se guardaron {len(dataset_array)} imágenes en la carpeta '{carpeta_salida}'.")

Cargando '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/turtles128.sav'... (Esto puede tardar un momento)
¡Archivo cargado! Se encontraron 17456 imágenes.
Guardando imágenes en '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'...
¡Listo! Se guardaron 17456 imágenes en la carpeta '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'.


Data augmentation

In [1]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
from tqdm import tqdm
from itertools import product, cycle

# Configuración
carpeta = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Mariquita"
num_a_generar = 16000
extensiones = (".jpg", ".jpeg", ".png")

# Definir todas las transformaciones posibles
TRANSFORMACIONES = {
    'flip_horizontal': lambda img: img.transpose(Image.FLIP_LEFT_RIGHT),
    'flip_vertical': lambda img: img.transpose(Image.FLIP_TOP_BOTTOM),
    'rotate_15': lambda img: img.rotate(15, expand=True),
    'rotate_-15': lambda img: img.rotate(-15, expand=True),
    'rotate_30': lambda img: img.rotate(30, expand=True),
    'rotate_-30': lambda img: img.rotate(-30, expand=True),
    'brightness_low': lambda img: ImageEnhance.Brightness(img).enhance(0.7),
    'brightness_high': lambda img: ImageEnhance.Brightness(img).enhance(1.3),
    'contrast_low': lambda img: ImageEnhance.Contrast(img).enhance(0.8),
    'contrast_high': lambda img: ImageEnhance.Contrast(img).enhance(1.2),
    'saturation_low': lambda img: ImageEnhance.Color(img).enhance(0.7),
    'saturation_high': lambda img: ImageEnhance.Color(img).enhance(1.3),
    'sharpness': lambda img: ImageEnhance.Sharpness(img).enhance(1.5),
    'blur': lambda img: img.filter(ImageFilter.GaussianBlur(radius=1)),
    'autocontrast': lambda img: ImageOps.autocontrast(img),
}

def generar_combinaciones():
    """Genera combinaciones únicas de transformaciones"""
    transformaciones_keys = list(TRANSFORMACIONES.keys())
    combinaciones = []
    
    # Transformaciones individuales
    for key in transformaciones_keys:
        combinaciones.append([key])
    
    # Combinaciones de 2 transformaciones
    for i, key1 in enumerate(transformaciones_keys):
        for key2 in transformaciones_keys[i+1:]:
            # Evitar combinaciones incompatibles
            if not (('flip' in key1 and 'flip' in key2) or 
                    ('rotate' in key1 and 'rotate' in key2) or
                    ('brightness' in key1 and 'brightness' in key2) or
                    ('contrast' in key1 and 'contrast' in key2)):
                combinaciones.append([key1, key2])
    
    # Combinaciones de 3 transformaciones (limitadas)
    categorias = {
        'geometrica': ['flip_horizontal', 'flip_vertical', 'rotate_15', 'rotate_-15'],
        'color': ['brightness_low', 'brightness_high', 'saturation_low', 'saturation_high'],
        'filtro': ['sharpness', 'blur', 'autocontrast', 'contrast_low', 'contrast_high']
    }
    
    for geo in categorias['geometrica'][:2]:
        for color in categorias['color'][:2]:
            for filtro in categorias['filtro'][:2]:
                combinaciones.append([geo, color, filtro])
    
    random.shuffle(combinaciones)
    return combinaciones

def aplicar_transformaciones(img, transformaciones_list):
    """Aplica una lista de transformaciones a una imagen"""
    for trans_key in transformaciones_list:
        img = TRANSFORMACIONES[trans_key](img)
    return img

# Obtener imágenes originales
imagenes = [f for f in os.listdir(carpeta) if f.lower().endswith(extensiones)]
print(f"📸 Imágenes originales: {len(imagenes)}")

if len(imagenes) == 0:
    print("❌ No se encontraron imágenes en la carpeta especificada.")
    exit()

# Generar combinaciones de transformaciones
combinaciones = generar_combinaciones()
print(f"🔄 Combinaciones de transformaciones generadas: {len(combinaciones)}")

# Crear iterador cíclico que repite las combinaciones si es necesario
iterador_combinaciones = cycle(combinaciones)

# Generar nuevas imágenes
contador_exitosos = 0
for i in tqdm(range(num_a_generar), desc="Generando imágenes"):
    archivo_original = random.choice(imagenes)
    ruta_original = os.path.join(carpeta, archivo_original)
    
    # Obtener siguiente combinación de transformaciones
    trans_actual = next(iterador_combinaciones)
    
    try:
        img = Image.open(ruta_original)
        
        # Convertir a RGB si es necesario (por ejemplo, PNG con transparencia)
        if img.mode in ('RGBA', 'LA', 'P'):
            img = img.convert('RGB')
        
        # Aplicar transformaciones
        img = aplicar_transformaciones(img, trans_actual)
        
        # Guardar con nombre descriptivo
        nombre, ext = os.path.splitext(archivo_original)
        trans_nombre = '_'.join(trans_actual[:2])  # Primeras 2 transformaciones en el nombre
        nuevo_nombre = f"{nombre}_aug{i}_{trans_nombre}{ext}"
        ruta_nueva = os.path.join(carpeta, nuevo_nombre)
        
        img.save(ruta_nueva, quality=95)
        contador_exitosos += 1
        
    except Exception as e:
        print(f"\n⚠️ Error con {archivo_original}: {e}")

print(f"\n✅ Se generaron {contador_exitosos} imágenes nuevas exitosamente.")
print(f"📁 Total de imágenes en carpeta: {len(imagenes) + contador_exitosos}")

📸 Imágenes originales: 6626
🔄 Combinaciones de transformaciones generadas: 117


Generando imágenes:   5%|▍         | 797/16000 [00:24<13:30, 18.75it/s]


⚠️ Error con ladybug_02797.jpg: image file is truncated (0 bytes not processed)


Generando imágenes:   6%|▌         | 933/16000 [00:32<06:53, 36.45it/s]/home/yisus/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Generando imágenes:   9%|▉         | 1449/16000 [00:49<05:37, 43.06it/s]


⚠️ Error con ladybug_02797.jpg: image file is truncated (0 bytes not processed)


...


⚠️ Error con ladybug_04502.jpeg: cannot identify image file '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Mariquita/ladybug_04502.jpeg'


Generando imágenes:  88%|████████▊ | 14123/16000 [07:52<01:02, 29.88it/s]


⚠️ Error con ladybug_01005.jpg: broken data stream when reading image file


Generando imágenes: 100%|██████████| 16000/16000 [09:25<00:00, 28.30it/s]


✅ Se generaron 15995 imágenes nuevas exitosamente.
📁 Total de imágenes en carpeta: 22621
